In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

**10.1- Exploração do banco de dados**
 
 **Assim como foi feito na competição Titanic, é necessário conhecer os banco de dados
 antes que se possa limpar os dados. Nesta sessão será realizada uma exploração a fim de
 entender melhor o banco de dados e identificar possíveis falhas que possam afetar o
 treinamento do modelo.**
 
 **1. O primeiro passo é importar e visualizar o banco de dados, rastrear os valores
 faltantes e identificar os momentos estatísticos existentes:**

**a. Importar o banco de dados utilizando o pandas;**

**b. Identificar o número de valores faltantes em cada colunas;**
 
**i.Qual sua hipótese para a existência desses valores faltantes?**
 
**c. Gerar momentos estatísticos do banco de dados;**
 
**i. Qual coluna parece fora do comum? Por que?**
 
**d. Utilizando o método “.hist” do pandas realize plots de distribuição dos valores
 das colunas:**
 
**i. É possível identificar algo de estranho com a coluna “Pressure(millibars)”. Identifique e explique o que há de errado;**

**ii. Utilizando “.loc” do bandas, identifique a quantidade de valores errados.**
 

In [ ]:
#letra a:

import pandas as pd

# a) Importar o banco
df = pd.read_csv("/kaggle/input/szeged-weather/weatherHistory.csv")

# Visualizar as 5 primeiras linhas
df.head()

In [ ]:
#letra b,i
# b) Contar valores faltantes por coluna
missing_values = df.isnull().sum()
print(missing_values)


**Como o Dataset é meteorológico os valores costumam falta por falhas temporárias de sensores, perda de conexão com a estação de coleta ou problemas de armazenamento e transmissão dos dados**

In [ ]:
#letra c,i:

# c) Gerar estatísticas descritivas
stats = df.describe()
stats

**Colunas como Pressure ou Temperature(C) podem se destacar se apresentarem valores fora da faixa física esperada, uma coluna fora do comum seria Pressure se apresentar valores muito baixos como  (< 870) ou muito altos como (> 1085)**

In [ ]:
#letra d,i:

import matplotlib.pyplot as plt

# d) Plotar histogramas de todas as colunas numéricas
df.hist(bins=30, figsize=(15, 10))
plt.tight_layout()
plt.show()

**Observação: se a distribuição mostrar picos artificiais, valores zeros ou negativos, isso sugere erro de medição ou registro incorreto.**

In [ ]:
#letra d, ii:

# Definir limites plausíveis para pressão atmosférica ao nível do mar
min_pressure = 870
max_pressure = 1085

# ii) Selecionar linhas com valores fora dessa faixa
pressure_errors = df.loc[(df["Pressure (millibars)"] < min_pressure) | 
                         (df["Pressure (millibars)"] > max_pressure)]

# Exibir quantidade de erros
len(pressure_errors)

**Isso indica que 1288 registros da sua base estão com valores de pressão fora da faixa que definimos como fisicamente plausível (870 a 1085 milibares).**


**2. Deve-se agora definir quais são as features categóricas e quais são as numéricas,
 para isso existe o método “.select_dtypes”, aqui será necessário identificar apenas os
 nomes das features.**

**a. Identifique as features numéricas, aquelas que são do tipo “int64” ou “float64”,
 utilizando “.select_dtypes”;**

**b. Identifique as features categóricas, aquelas que são do tipo “object”;**

**i.Uma das features é do tipo object mas não é categórica. Qual é ela?**

**c. Inspecione os valores únicos de cada feature categórica, para garantir que não
 há nenhum valor que ocorra apenas uma vez, o que iria interferir na separação
 do banco de dados em treino e validação (Por que?). Utilize o método
 ".value_counts" para isso;**

**i.Há uma feature em que existem valores que ocorrem mais de uma vez.
 esse valores serão retirados do banco de dados futuramente;**

In [ ]:
#letra a:

#- select_dtypes(include=[...]) → filtra colunas pelos tipos especificados.
#- .columns → retorna apenas o nome das colunas.
# Features numéricas
numeric_features = df.select_dtypes(include=["int64", "float64"]).columns
print("Features numéricas:", list(numeric_features))

In [ ]:
#Features categóricas:
# Features categóricas (tipo object)
categorical_features = df.select_dtypes(include=["object"]).columns
print("Features categóricas:", list(categorical_features))

In [ ]:
#letra b,i:

# Checar colunas object que não são realmente categóricas
for col in categorical_features:
    print(col, type(df[col].iloc[0]))

In [ ]:
# Converte para datetime garantindo fuso horário UTC
df["Formatted Date"] = pd.to_datetime(df["Formatted Date"], utc=True)

In [ ]:
#letra c:
#Inspecionar valores únicos em cada feature categórica:
for col in categorical_features:
    print(f"\nColuna: {col}")
    print(df[col].value_counts())

In [ ]:
#letra c,i:

# Exemplo: filtrar valores únicos de uma coluna categórica
rare_values = df["Summary"].value_counts()[df["Summary"].value_counts() == 1]
print("Valores raros na coluna Summary:", rare_values)

**1. Uma das features do tipo “object” é “Formatted Date” que, na verdade, indica a data
 do registro climático. Essa coluna deverá então ser transformada para o formato de
 data para que seja possível extrair informações úteis para o treinamento do modelo.**

 **a. Existe um valor nessa feature que indica o fuso-horário (“+0200”). Para que a
 coluna seja corretamente transformada este valor desse ser retirado;**

**i.Deve ser realizado um processo semelhante ao item 9.2-1, em que
 será utilizado o método “str.split” para retirar o texto “+0200” de todos
 os valores da feature;**

**ii.Use o método "to_datetime" para transformar a coluna no formato
 desejado.**

**b. Utilizando os métodos existentes em “.dt” crie novas colunas retirando as
 informações de hora, dia, mês e ano de cada valor de data na feature;**
 
 **i. Utilize os métodos “dt.hour ”, “dt.day ”, “dt.month ”, “dt.year ”;**

 **ii. Retire a coluna original de data do banco de dados.**

 

In [ ]:
# a.i) Remover timezone do final da string (cobre +0200, +0000, +00:00, -0300, Z etc.)
# - Converte para string para garantir o uso do .str
# - Usa um único split com regex que pega o sufixo de timezone no final da linha
df["Formatted Date"] = (
    df["Formatted Date"]
    .astype(str)
    .str.split(r"(?:\s*[+-]\d{2}:?\d{2}|Z)$")  # timezone no fim (com ou sem espaço e com ou sem dois-pontos)
    .str[0]
    .str.strip()
)

# a.ii) Converter para datetime (deixe o pandas inferir o formato)
df["Formatted Date"] = pd.to_datetime(df["Formatted Date"])

In [ ]:
# b.i) Criar as novas colunas a partir da data
df["hour"] = df["Formatted Date"].dt.hour
df["day"] = df["Formatted Date"].dt.day
df["month"] = df["Formatted Date"].dt.month
df["year"] = df["Formatted Date"].dt.year

# b.ii) Remover a coluna original
df.drop(columns=["Formatted Date"], inplace=True)

In [ ]:
#Checagem rápida:

print(df[["hour", "day", "month", "year"]].head())
print(df[["hour", "day", "month", "year"]].describe())

**2. Agora será criada a pipeline para completar a limpeza dos dados, com ela será
 realizada a substituição dos valores faltantes nas colunas “Pressure (millibars)” e
 “Precip Type” e o encoding, utilizando ordinal encoder, das variáveis categóricas. Para
 isso será utilizada a biblioteca Column Transformer do sklearn, mas antes deverão ser
 retirados os dados que não servirão para o treinamento do modelo.**
 
 **a. Retirar a coluna “Loud Cover”, já que contém apenas valores nulos, e linhas
 cujo valor da coluna “Summary” aparecem apenas uma vez;**
 
 **i.Para localizar as linhas que serão retiradas utilize o método “.loc”.**
 
**b. Definir imputers para variáveis numéricas e para categóricas, e o encoder das
 variáveis categóricas;**
 
 **i.Configurar imputer das variáveis numéricas com simple imputer,
 utilizando a média como substituta do valor faltante e indicando que o
 valor 0 significa valor faltante (valores iguais a 0 em “Pressure
 (millibars)” representam valores faltantes);**
 
 **ii.Criar uma pipeline para tratamento das variáveis categóricas, primeiro
 passo deve ser a substituição dos valores faltantes utilizando simple
 imputer, substituindo o valor faltante por um texto, o segundo passo é
 realizar o encoding utilizando ordinal encoder.**
 
 **c. Criar um objeto do tipo Column Transformer para aplicar os passos criados
 anteriormente.**
 
 **i.Configurar column transformer para fazer o tratamento das variáveis
 numéricas e categóricas.**

In [ ]:
print(df.columns.tolist())

In [ ]:
#A coluna Loud Cloud não existe, pelo que foi visto no código anterior, então façamos o seguinte:

# a) (adaptado) — só remove se existir
if "Loud Cover" in df.columns:
    df.drop(columns=["Loud Cover"], inplace=True)

# a.i) Remover linhas em que 'Summary' aparece apenas uma vez
linhas_unicas = df["Summary"].value_counts() == 1
valores_unicos_summary = linhas_unicas[linhas_unicas].index

df = df.loc[~df["Summary"].isin(valores_unicos_summary)]

In [ ]:
#b-definir imputer e encoder:

from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OrdinalEncoder
from sklearn.pipeline import Pipeline

In [ ]:
#b.i.Pipeline para variávei numéricas:

# Lista de colunas numéricas (refaça a seleção se já não tiver guardada)
num_cols = df.select_dtypes(include=["int64", "float64"]).columns

# Substituir zeros em 'Pressure (millibars)' por NaN
import numpy as np
df["Pressure (millibars)"] = df["Pressure (millibars)"].replace(0, np.nan)

# Imputer numérico usando média
numeric_transformer = SimpleImputer(strategy="mean")

In [ ]:
#Pipeline para variáveis categóricas:

# Lista de colunas categóricas
cat_cols = df.select_dtypes(include=["object"]).columns

# Pipeline para categóricas: imputação + encoding
categorical_transformer = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="constant", fill_value="Desconhecido")),
    ("encoder", OrdinalEncoder(handle_unknown="use_encoded_value", unknown_value=-1))
])

In [ ]:
#Criar ColumnTransformer:

from sklearn.compose import ColumnTransformer

preprocessor = ColumnTransformer(
    transformers=[
        ("num", numeric_transformer, num_cols),
        ("cat", categorical_transformer, cat_cols)
    ]
)

**3. O último passo da limpeza dos dados será a separação em treino e validação,
 aplicação da pipeline de limpeza e o encoding da coluna que será o target (“Daily
 summary”).**
 
**a. Definir quais são as features, o que é o target e separar o banco de dados
 utilizando train-test split, deve-se garantir que todos os valores do target estão
 distribuídos de forma equilibrada na reparação do banco de dados;**
 
 **i.Indicar o target como sendo “Daily Summary” e as features como as
 colunas restantes;**
 
 **ii. Utilizar train-test split, configurado para que o tamanho do banco de
 dados de teste seja 30% do banco de dados original e utilizando o
 target como parâmetro de “stratify”;**
 
 **b. Aplicar pipeline de limpeza do banco de dados;**
 
 **i.Utilize apenas as features dentro da pipeline, ajuste utilizando as
 features de treino e apenas transforme as de teste.**
 
 **c. Utilize label encoder para realizar o encoding do target.**
 
 **i.Da mesma forma, utilize o target de treino para ajustar e apenas
 transforme o de teste.**

In [ ]:
from sklearn.model_selection import train_test_split

# a.i) Definindo target e features
target = "Daily Summary"
X = df.drop(columns=[target])  # todas as colunas exceto o target
y = df[target]                  # coluna alvo

# a.ii) Dividindo treino e teste com estratificação no target
X_train, X_test, y_train, y_test = train_test_split(
    X,
    y,
    test_size=0.30,         # 30% para teste
    stratify=y,             # mantém distribuição proporcional das classes
    random_state=42         # reprodutibilidade
)

In [ ]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer

# Exemplo: identifica colunas numéricas e categóricas do X (sem target)
num_features = X.select_dtypes(include=['int64', 'float64']).columns
cat_features = X.select_dtypes(exclude=['int64', 'float64']).columns

# Pipelines para cada tipo
numeric_pipeline = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="median")),
    ("scaler", StandardScaler())
])

categorical_pipeline = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="most_frequent")),
    ("encoder", OneHotEncoder(handle_unknown="ignore"))
])

# ColumnTransformer só com features — target fica de fora
preprocessor = ColumnTransformer(transformers=[
    ("num", numeric_pipeline, num_features),
    ("cat", categorical_pipeline, cat_features)
])

In [ ]:
X_train_processed = preprocessor.fit_transform(X_train)
X_test_processed = preprocessor.transform(X_test)

In [ ]:
#c-Encoding do target com LabelEncoder

from sklearn.preprocessing import LabelEncoder

# Criar encoder do target
label_encoder = LabelEncoder()

# Ajustar no treino e transformar
y_train_encoded = label_encoder.fit_transform(y_train)

# Apenas transformar o teste
y_test_encoded = label_encoder.transform(y_test)

In [ ]:
#Cheacegem rápida:

print("Shape treino:", X_train_processed.shape, "Target:", len(y_train_encoded))
print("Shape teste:", X_test_processed.shape, "Target:", len(y_test_encoded))
print("Classes do target:", label_encoder.classes_)

**10.3- Treinamento e Avaliação da rede neural**

**Após realizada a limpeza de dados, o último passo será construir a rede neural,
 treiná-la e avaliá-la. Para isso será utilizado o framework Keras, ao final do treinamento serão
 mostrados gráficos que indiquem a evolução da acurácia e do erro, calculado por cross entropy, em relação ao número de épocas utilizadas para o treinamento. Outro passo
 importante é a indicação para o modelo de que o target é um valor discreto, para isso será
 utilizada uma ferramenta nativa do keras que transforma o target de forma semelhante ao
 one-hot encoding.**

**1. Primeiramente deve-se criar a arquitetura da rede neural, ela terá duas camadas
 escondidas, sendo cada uma com uma saída de dimensão 256, entre elas serão
 utilizados Batch normalization e Dropout. Como funções de ativação serão usados
 “relu” para as camadas escondidas e o “softmax” na camada de saída, esta função
 serve para que o modelo realize predições na forma de classificação não binária, ou
 seja, que seja capaz de prever mais de duas classes.**
 
 **a. Configurar o tamanho da entrada, que é o número de features para
 treinamento, e o tamanho da saída, que será a quantidade de classes do
 target;**
 
 **i.Utilizar o método “.shape” para descobrir quais as dimensões das
 features e “.unique” para descobrir quantas classes existem no target.**
 
 **b. Construir arquitetura do modelo utilizando a descrição já informada;**
 
**i.Camada de entrada utilizando batch normalization configurando seu
 tamanho a partir das features do banco de dados;**
 
 **ii.Primeira camada escondida utilizando “relu” como função de ativação,
 saída de dimensão 256, batch normalization e dropout de 0.3;**
 
 **iii.Segunda camada escondida igual à primeira;**
 
 **iv.Camada de saída com tamanho igual ao número de classes do target e
 função de ativação “softmax”.**
 
 **c. Realizar tratamento do target utilizando o método “utils.to_categorical” do
 keras.**
 
 **i.Transformar target de treino e de teste utilizando o método do keras;**
 
 **ii.Veja as dimensões da matriz de target e responda, o target é composto
 de quantas classes (Possíveis previsões)?**
 
 **2. Configurar o otimizador, a função de perda que será utilizada e a métrica para
 avaliação do modelo.**

 **a. Como otimizador será utilizado “adam”, como função de perda
 “categorical_crossentropy”  e como métrica para avaliação “categorical_accuracy”.**
 
 **3. Por fim, será realizado o treinamento do modelo e plotagem dos resultados das
 avaliações, será utilizado o early stopping para evitar o overfitting. Os hiperparâmetros
 que serão utilizados, como o número de épocas e tamanho de cada batch durante o
 treinamento do modelo podem ser alterados pelo aluno para teste e possível
 aprimoramento.**
 
**a. Configurar early stopping;
 i.Utilizar a biblioteca do keras para configurar early stopping, com uma
 tolerância de 10 épocas e uma variação mínima no resultado da função
 de perda de 0.001.**
 
 **b. Realizar o treino e armazenar os resultados da avaliação do modelo;**
 
 **i. Utilizar as features e target de treino para treinar o modelo;**

 **ii. Utilizar os dados de teste para validar o modelo;**

 **iii. Utilizar um tamanho de batch de 3000;**

 **iv. Utilizar 100 épocas para treino;**

 **v. Indicar utilização de early stopping;**

**c. Para a plotagem deve-se transformar os resultados do treino em um dataframe
 do pandas e plotar os valores de perda e acurácia em função das épocas.**
 
 **i.Utilizar o método “.DataFrame” para transformar resultados em data
 frame do pandas;**
 
 **ii.Utilizar gráfico de linha para plotar os resultados da perda e da acurácia
 emfunção das épocas de treino**
 

In [ ]:
#Criar uma arquitetura de rede neural:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"
import numpy as np
import pandas as pd
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.utils import to_categorical

# a.i) Tamanho da entrada e saída
input_dim = X_train_processed.shape[1]      # nº de features
num_classes = len(np.unique(y_train_encoded))  # nº de classes no target

print(f"Número de features (input_dim): {input_dim}")
print(f"Número de classes (output_dim): {num_classes}")

# b) Construção do modelo
model = keras.Sequential()

# b.i) Camada de entrada com Batch Normalization
model.add(layers.Input(shape=(input_dim,)))
model.add(layers.BatchNormalization())

# b.ii) Primeira camada escondida
model.add(layers.Dense(256, activation="relu"))
model.add(layers.BatchNormalization())
model.add(layers.Dropout(0.3))

# b.iii) Segunda camada escondida
model.add(layers.Dense(256, activation="relu"))
model.add(layers.BatchNormalization())
model.add(layers.Dropout(0.3))

# b.iv) Camada de saída com softmax
model.add(layers.Dense(num_classes, activation="softmax"))

In [ ]:
#1.c)Tratamento do target(One-Hot Encoding com to_categorical)


# i) Transformar target de treino e teste
y_train_cat = to_categorical(y_train_encoded, num_classes=num_classes)
y_test_cat = to_categorical(y_test_encoded, num_classes=num_classes)

# ii) Checar dimensões
print("Formato y_train_cat:", y_train_cat.shape)
print("Formato y_test_cat:", y_test_cat.shape)
print(f"O target possui {y_train_cat.shape[1]} classes possíveis.")



In [ ]:
#2)Configurar otimizador, função de perda e métrica:

model.compile(
    optimizer="adam",
    loss="categorical_crossentropy",
    metrics=["categorical_accuracy"]
)

In [ ]:
#3)Treinamento com Early Stopping:

# a) Configurar EarlyStopping
early_stop = keras.callbacks.EarlyStopping(
    monitor="val_loss",
    patience=10,
    min_delta=0.001,
    restore_best_weights=True
)

# b) Treinar o modelo
history = model.fit(
    X_train_processed,
    y_train_cat,
    validation_data=(X_test_processed, y_test_cat),
    batch_size=3000,
    epochs=100,
    callbacks=[early_stop],
    verbose=1
)

In [ ]:
#c)Plotar evolução de perda e acurácia:

import matplotlib.pyplot as plt

# Transformar histórico em DataFrame
hist_df = pd.DataFrame(history.history)

# Plotar Perda
plt.figure(figsize=(12,5))

plt.subplot(1, 2, 1)
plt.plot(hist_df["loss"], label="Treino")
plt.plot(hist_df["val_loss"], label="Validação")
plt.xlabel("Épocas")
plt.ylabel("Perda (Cross-Entropy)")
plt.title("Evolução da Perda")
plt.legend()

# Plotar Acurácia
plt.subplot(1, 2, 2)
plt.plot(hist_df["categorical_accuracy"], label="Treino")
plt.plot(hist_df["val_categorical_accuracy"], label="Validação")
plt.xlabel("Épocas")
plt.ylabel("Acurácia")
plt.title("Evolução da Acurácia")
plt.legend()

plt.tight_layout()
plt.show()